In [2]:
# # !pip3 install -q -U huggingface-hub==0.13.0
# !pip3 install -q -U graphviz==0.20.1
# !pip3 install -q torchview==0.2.6

In [3]:
import sys
sys.path = ['../input/nar-ai4code-utils'] + sys.path

In [4]:
# import graphviz
# from torchview import draw_graph
# graphviz.set_jupyter_format('png')

In [5]:
import utils
import preprocess

In [6]:
import pandas as pd
import torch
import random
import os
# from utils import create_label
import numpy as np

# coding=utf-8
import numpy as np
import pandas as pd
import os, random, time, gc, argparse, shutil
from datetime import datetime
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.optim.lr_scheduler import _LRScheduler, ReduceLROnPlateau, CosineAnnealingLR, CosineAnnealingWarmRestarts
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset
from torch.utils.data.sampler import Sampler
from transformers import get_constant_schedule_with_warmup, AdamW, get_cosine_schedule_with_warmup
from transformers import AutoTokenizer
from transformers import AutoConfig, AutoModel, AutoModelForMaskedLM
import warnings

## Config

In [7]:
parser = argparse.ArgumentParser(description='PyTorch')
parser.add_argument('--out_dir', default='', type=str, help='destination where trained network should be saved')
parser.add_argument('--gpu_id', default='0', type=str, help='gpu id used for training')
parser.add_argument('--model_name', default='deberta-v3-base', type=str)
parser.add_argument('--base_epoch', default=1, type=int, help='number of total epochs to run')#30
parser.add_argument('--batch_size', default=2, type=int, help='train mini-batch size')# 8
parser.add_argument('--workers', default=2, type=int, help='number of data loading workers (default: 4)')
parser.add_argument('--lr', default=17e-6, type=float, help='learning rate') # 5e-6
parser.add_argument('--eta_min', default=11e-6, type=float, help='learning rate eta_min') # 5e-6

# gradient accumulation steps as the batch_size=1 then set this to 5.
parser.add_argument('--n_accumulate', default=5, type=int) 

parser.add_argument('--max_grad_norm', default=-1, type=float) # 1.0
parser.add_argument('--weight_decay', default=0.0, type=float)
parser.add_argument('--folds', default='', type=str)
parser.add_argument('--pre_epoch', default=15, type=int)

# args = parser.parse_args()
args, unknown = parser.parse_known_args()

In [8]:
class Parameter(object):
    def __init__(self):
        # data
        self.result_dir = './models/' # ./user_data/
        self.result_dir_1 = '../input/nar-ai4code-models/'        
        self.data_dir = './'
        self.model_dir = '../input/nar-transformers/'
        self.k_folds = 5
        self.random_seed = 27
        self.seq_length = 512 # 512; sum of "cell length" for a particular id.
        self.seq_max_length = 4096  
        self.cell_max_length = 128 # max length of a cell.
        self.cell_count = 128        
        self.device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
        # model
        self.use_cuda = torch.cuda.is_available()
        self.gpu = 0
        self.print_freq = 500
        self.weight_decay = 0
        self.optim = 'Adam'

    def get(self, name):
        return getattr(self, name)

    def set(self, **kwargs):
        for k, v in kwargs.items():
            setattr(self, k, v)

    def __str__(self):
        return '\n'.join(['%s:%s' % item for item in self.__dict__.items()])
    
parameter = Parameter()
parameter.set(**args.__dict__)

In [9]:
def seed_everything(random_seed):
    random.seed(random_seed)
    np.random.seed(random_seed)
    torch.manual_seed(random_seed)
    os.environ["PYTHONHASHSEED"] = str(random_seed)
#     if torch.cuda.is_available():
    torch.cuda.manual_seed(random_seed)
    torch.cuda.manual_seed_all(random_seed)
    #         torch.backends.cudnn.enabled = False
    torch.backends.cudnn.benchmark = False        
    torch.backends.cudnn.deterministic = True

## Classes

In [10]:
class MarkdownDataset(Dataset):

    def __init__(self, meta_data: pd.DataFrame, tokenizer, fold: int = -1, mode='train', parameter=None):
        self.meta_data = meta_data.copy()
        if mode == 'train':
            pass
            self.meta_data = self.meta_data[self.meta_data['fold_flag'] != fold].copy()
#             self.meta_data = self.meta_data.iloc[:60000]
            self.meta_data.reset_index(drop=True, inplace=True)
        elif mode == 'valid':
            self.meta_data = self.meta_data[self.meta_data['fold_flag'] == fold].copy()
#             self.meta_data = self.meta_data[self.meta_data['id'].isin(self.meta_data['id'].values[:1000])]
            self.meta_data.reset_index(drop=True, inplace=True)
        elif mode == 'test':
            pass
        else:
            raise ValueError(mode)

        if tokenizer.sep_token != '[SEP]':
            self.meta_data['source'] = self.meta_data['source'].apply(
                lambda x: [
                    y.replace(tokenizer.sep_token, '').replace(tokenizer.cls_token, '').replace(tokenizer.pad_token, '')
                    for y in x])
        self.parameter = parameter        
        self.seq_max_length = self.parameter.seq_max_length
        self.source = self.meta_data['source'].values
        self.cell_type = self.meta_data['cell_type'].values
        self.batch_max_length = self.meta_data['batch_max_length'].values        
        # self.cell_id = self.meta_data['cell_id'].values
        # self.rank = self.meta_data['rank'].values
        # self.dense_features = self.meta_data[['cell_count','markdown_count', 'code_count']].values
        self.mode = mode
        self.tokenizer = tokenizer

    def __getitem__(self, index):
        source = self.source[index]
            # (no data cleaning) source =>
            # array([' this python 3 environment .... the current session', ..., ' на обучении на один признак больше чем на тесте '], dtype=object)
            
            # [len(x) for x in source] => 
            # [845, 491, 47, 40, 555, 154, 54, 15, 141, 51, 74, 35, ..., 62, 34, 39, 49]            
        
            # max([len(x) for x in source]), min([len(x) for x in source]) => 3722, 10
        cell_type = self.cell_type[index]
            # cell_type =>
            # array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ....,  1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])        
        
#         rank = self.rank[index]
#             # rank => array([0.01724138, 0.05172414, 0.0862069 , 0.12068966, 0.15517241, ..., 0.93103448, 0.62068966, 0.4137931 ]) 
            
        batch_max_len = min(self.batch_max_length[index], self.seq_max_length)            
    
        # dense_features = 1#self.dense_features[index]
        # if self.mode == 'train':
        #     range_tmp1 = [ i for i in range(len(cell_type)) if cell_type[i]==0]
        #     range_tmp2 = [ i for i in range(len(cell_type)) if cell_type[i]==1]
        #     np.random.shuffle(range_tmp2)
        #     source = [source[i] for i in range_tmp1 + range_tmp2]
        #     rank = [rank[i] for i in range_tmp1 + range_tmp2]
        #     rank2 = [rank2[i] for i in range_tmp1 + range_tmp2]

        cell_inputs = self.tokenizer(#.batch_encode_plus
            source,#.tolist(),
            add_special_tokens=False,
            max_length=self.parameter.cell_max_length,
            # padding="max_length",
            return_attention_mask=False,
            truncation=True,
        )
            # cell_inputs =>
            # {'input_ids': [[291, 24233, 404, ..., 2, 763, 5514, 4219], ..., [15003, 42120, 69819, ..., 15003, 65639, 109249]],
            # 'token_type_ids': [[0, 0, 0, ..., 0, 0, 0, 0], ..., [0, 0, 0, ..., 0, 0, 0]]}

            # [len(x) for x in cell_inputs['input_ids']] => [128, 105, 7, 10, 127, 28, 12, 4, 26, 12, 23, 9, ... , 18]        
            # max([len(x) for x in cell_inputs['input_ids']]), min([len(x) for x in cell_inputs['input_ids']]) => 128, 4        
        
        seq, seq_mask, target_mask = self.max_length_rule_base(cell_inputs['input_ids'], cell_type, batch_max_len)# rank)
        
        # if self.mode == 'train':
        #     attention_mask, target = self.random_mask(attention_mask, target)
        return seq, seq_mask, target_mask
        # return encoded['input_ids'][0], encoded['attention_mask'][0], np.array(target, dtype=np.float32)

    def __len__(self):
        return len(self.meta_data)

    def max_length_rule_base(self, cell_inputs, cell_type, batch_max_len):# rank):
        init_length = [len(x) for x in cell_inputs]
            # init_length => [128, 105, 7, ..., 13, 16, 18]
            # batch_max_len => 4096 
            # len(init_length) => 128

        total_max_length = batch_max_len - len(init_length)
            # total_max_length => 3968           
        min_length = total_max_length // len(init_length)
            # min_length => 31
                         
        cell_length = self.search_length(init_length, min_length, total_max_length, len(init_length))
            # cell_length => [51, 51, 51, 51, 41, 51, 51, 51, 51, 51, 42, 51, 11, 51, 51, 51 ....
            # len(cell_length) => 128

        seq = []
        for i in range(len(cell_length)):
            if cell_type[i] == 0:
                seq.append(self.tokenizer.cls_token_id)
                # self.tokenizer.cls_token_id => 1 
            else:
                seq.append(self.tokenizer.sep_token_id)
                # self.tokenizer.sep_token_id => 2
            if cell_length[i] > 0:
                seq.extend(cell_inputs[i][:cell_length[i]])

            # cell_inputs[i] => [291, 24233, 404, ..., 2, 763, 5514, 4219]
            # seq => [1, 291, 24233, 404, 1192, 699, 275, 386, 2136, 7027, 7296, 2627] 
            
        # seq => [1, 291, 24233, 404, 1192, 699, 275, 386, 2136, 7027, 7296, 2627, 1, 6306, ..., 2, 15003, ..., 88580]
        
        # seq_mask or attention_mask tells model which position should it attend in seq. 1 -> attend and 0 -> not attend.
        # tokenizer.pad_token_id => 0
        if len(seq) < batch_max_len:
            seq_mask = [1] * len(seq) + [0] * (batch_max_len - len(seq))
            seq = seq + [self.tokenizer.pad_token_id] * (batch_max_len - len(seq))
        else:
            seq_mask = [1] * batch_max_len
            seq = seq[:batch_max_len]
            
        # seq_mask is attention_mask.
        seq, seq_mask = np.array(seq, dtype=np.int32), np.array(seq_mask, dtype=np.int32)
        target_mask = np.where((seq == self.tokenizer.cls_token_id) | (seq == self.tokenizer.sep_token_id), 1, 0) 
            # target_mask => [1 0 0 0 0 0 0 0 0 0 0 0 1 ..... 0 0 0 0]            
            
#         target = np.zeros(len(seq), dtype=np.float32)
        
#         # return indices where self.tokenizer.cls_token_id or self.tokenizer.sep_token_id is present
#         tmp = np.where((seq == self.tokenizer.cls_token_id) | (seq == self.tokenizer.sep_token_id))
#             # tmp => (array([  0,  12,  24,  32,  43,  55,  67,  79,  ..., 479, 487, 495, 503]),)        
#         target[tmp] = rank
#             # target => [0.01724138 0.         0.         0.    ...      0.        ]
            
#         sample_weight = np.zeros(len(seq), dtype=np.float32)
#         sample_weight = np.where(seq == self.tokenizer.cls_token_id, 0.33, sample_weight)
#         sample_weight = np.where(seq == self.tokenizer.sep_token_id, 1.0, sample_weight)
#         dense_features = np.zeros(batch_max_len, dtype=np.float32)
#         dense_features[tmp] = rank2

        return seq, seq_mask, target_mask#, target

    # Static method objects provide a way of defeating the transformation of function objects to method objects.
    # A static method does not receive an implicit first argument. we can even call it from an instance. 
    # A class method receives the class as implicit first argument, just like an instance method receives the instance.
    # The insertion of @staticmethod before a method definition, then, stops an instance from sending itself as an argument.
    @staticmethod
    def search_length(init_length, min_length, total_max_length, cell_count, step=5, max_search_count=50):# step=4
            # init_length => [128, 128, 97, 68, 41, 60, 60, 119, 66, 127, ...
            # np.sum(init_length) => 6377
            # min_length, total_max_length, cell_count => 31, 3968, 128
            # len(init_length) is cell_count.
            
        if np.sum(init_length) <= total_max_length:
            return init_length

        res = [min(init_length[i], min_length) for i in range(cell_count)]      
            # res => [31, 31, 31, 31, 31, 31, 31, 31, 31, 31, ...
        for s_i in range(max_search_count):
            tmp = [min(init_length[i], res[i] + step) for i in range(cell_count)]# step=4
                # tmp => [36, 36, 36, 36, 36, 36, 36, 36, ...
                # np.sum(tmp) => 3234                                 
            if np.sum(tmp) < total_max_length:
                res = tmp
            else:
                break
                
        for s_i in range(cell_count):
            tmp = [i for i in res]
                # tmp => [46, 46, 46, 46, 41, 46, 46, 46, ...                 
            tmp[s_i] = min(init_length[s_i], res[s_i] + step)
                # tmp[s_i], np.sum(tmp) => 51, 3906                
            if np.sum(tmp) < total_max_length:
                res = tmp
            else:
                break
                
        # res (after all above execution) => [51, 51, 51, 51, 41, 51, 51, 51, 51 ...
        return res


In [11]:
class MarkdownModel(nn.Module):
    def __init__(self, name, num_classes=1, seq_length=96, pretrained=True):
        super(MarkdownModel, self).__init__()
        # self.encoder = AutoModel.from_pretrained(name, attention_probs_dropout_prob=0.1, hidden_dropout_prob=0.1)
        self.config = AutoConfig.from_pretrained(name)
        self.config.attention_probs_dropout_prob = 0.
        self.config.hidden_dropout_prob = 0.
        self.config.max_position_embeddings = 4096 * 2 
        # self.config.output_hidden_states = True
        if pretrained:
            self.encoder = AutoModel.from_pretrained(name, config=self.config, ignore_mismatched_sizes=True)
            # self.encoder = AutoModelForMaskedLM.from_pretrained(name, config=self.config)
        else:
            # self.encoder = AutoModelForMaskedLM.from_config(self.config)
            self.encoder = AutoModel.from_config(self.config)

        # self.encoder = AutoModel.from_pretrained(name)
        # print(self.encoder.__dict__)
        # transformer_layers = 2
        # self.seq_length = seq_length
        # self.transformer_layers = transformer_layers
        self.in_dim = self.encoder.config.hidden_size

#         self.pe = PositionalEncoding(self.in_dim)
#         self.trans = nn.Sequential(
#             *[TransformerBlock(emb_s=64, head_cnt=self.in_dim // 64, dp1=0., dp2=0.) for _ in
#               range(transformer_layers)])
        self.bilstm = nn.LSTM(self.in_dim, self.in_dim, num_layers=1, 
                              dropout=self.config.hidden_dropout_prob, batch_first=True,
                              bidirectional=True)
    
#         self.dropouts = nn.ModuleList([nn.Dropout(0.5) for _ in range(5)])
#         hidden = 64
#         dropout = 0.
#         self.sequence = nn.Sequential(
#             # nn.BatchNorm1d(1),
#             nn.Linear(1, hidden),  # todo
#             nn.Dropout(dropout),
#             nn.ReLU(),
#             # nn.BatchNorm1d(hidden),
#             nn.Linear(hidden, hidden),
#             nn.Dropout(dropout),
#             nn.ReLU()
#         )

        self.last_fc = nn.Linear(self.in_dim*2, num_classes)
        # self.fc = nn.LazyLinear(num_classes)
        
        # function is intended to be used to initialize neural network parameters, -
        # - so they all run in torch.no_grad() mode and will not be taken into account by autograd.
        # Fills the input Tensor with values drawn from the normal distribution N(mean,std_square).
        torch.nn.init.normal_(self.last_fc.weight, std=0.02)
        
        self.sig = nn.Sigmoid()

    def forward(self, x, mask):
        x = self.encoder(x, attention_mask=mask)["last_hidden_state"]
        # x = x.reshape(-1, code_count, self.seq_length, self.in_dim).mean(2)
        #         x = torch.sum(x * mask.unsqueeze(-1), dim=1) / torch.sum(mask, dim=1).unsqueeze(-1)
        #         x = x.reshape(-1, code_count, self.in_dim)
        # x = x + self.sequence(dense_features.unsqueeze(-1))

#         prev = None
#         x = self.pe(x)
#         for i in range(self.transformer_layers):
#             # x = x * mask.unsqueeze(-1)
#             x, prev = self.trans[i](x, prev)
        # x = torch.sum(x * mask.unsqueeze(-1), dim=1) / torch.sum(mask, dim=1).unsqueeze(-1)
        # x = torch.cat([x, self.sequence(dense_features.unsqueeze(1)).repeat(1,2048,1)], dim=2)
        # x = x.mean(1)
        x, _ = self.bilstm(x)
        out = self.last_fc(x)
#         for i, dropout in enumerate(self.dropouts):
#             if i == 0:
#                 out = self.last_fc(dropout(x))
#             else:
#                 out += self.last_fc(dropout(x))
#         out /= len(self.dropouts)
        # out = self.sig(out)
        out = out.squeeze(-1)
        return out

## Test

In [12]:
def validate(model, valid_loader, max_length=4096):
    # batch_time = AverageMeter()
    # losses = AverageMeter()
    
    # switch to evaluate mode
    model.eval()
    rank_pred = []
    masks = []
    for i, batch_data in enumerate(valid_loader):
        if parameter.use_cuda:
            batch_data = (t.cuda() for t in batch_data)
        seq, seq_mask, target_mask = batch_data
            # seq.shape, seq_mask.shape => torch.Size([1, 213]), torch.Size([1, 213])
        outputs = model(seq, seq_mask)
        outputs= outputs.detach().cpu().numpy()
            # outputs.shape => (1, 213) 
        target_mask = target_mask.detach().cpu().numpy()#.reshape((outputs.shape[0], -1))       
            # target_mask.shape => (1, 213)
        
        # taken 4096; different sequences have different shapes so taken maximum length -
        # - otherwise np.concatenate does not concat different shapes.
        tmp1 = np.zeros((outputs.shape[0], max_length)) 
            # tmp1.shape => (1, 4096)
        tmp1[:, :outputs.shape[1]] = outputs
        tmp2 = np.zeros((outputs.shape[0], max_length))
            # tmp2.shape => (1, 4096)
        tmp2[:, :outputs.shape[1]] = target_mask
        
        rank_pred.append(tmp1)
        masks.append(tmp2)

    
    rank_pred = np.concatenate(rank_pred)
        # rank_pred.shape (BS, 4096)
    masks = np.concatenate(masks)
        # masks.shape (BS, 4096)

    return rank_pred, masks

In [13]:
def get_preds(my_df, my_loader, my_model, model_path, max_length=4096):
    if my_df.shape[0] > 0:
        my_model.load_state_dict(torch.load(model_path)['state_dict'])
    if parameter.use_cuda:
        my_model = my_model.cuda()
    with torch.no_grad():
        y_pred, mask = validate(my_model, my_loader, max_length)
    return y_pred, mask

In [14]:
# https://www.kaggle.com/code/anyai28/fast-inference-by-padding-optimization
def get_sorted_test_df(df, extract_col, tokenizer, batch_size, cell_max_length=128):
    input_lengths = []
        # batch_size => 2
        # len(df[extract_col].values) => len of rows in train/test dataframe => 10
    for text in df[extract_col].values:        
            # text[0] => import numpy as np linear algebra import pandas as pd pd set option display ...        
        tmp = tokenizer(#.batch_encode_plus(
            text,#.tolist(),
            add_special_tokens=False,
            max_length=cell_max_length,
            return_attention_mask=False,
            truncation=True,
        )
            # tmp['input_ids'][0] => [6306, 67927, 283, 845, 407, 6306, 36221, 11751, 283, 76767]
        
        init_length = [len(x) for x in tmp['input_ids']]
            # init_length; lengths of cells  in particular input_id
            # init_length => [128, 3, 19, 118, 22, 45, 53, 36, 4, 36, 4, 14, 5 ...
        
        total_length = np.sum(init_length) + len(init_length)
            # total_length; sum of words plus "number of groups in which these words are divided" (for a particular id).
            # np.sum(init_length), len(init_length) => 2908, 93

        input_lengths.append(total_length)

    df['input_lengths'] = input_lengths
        # input_lengths => [3001, 213, 6505, 6223, 5624, 206, 1238, 482, 1052, 1617]
    length_sorted_idx = np.argsort([-l for l in input_lengths])
        # [-l for l in input_lengths] => [-3001, -213, -6505, -6223, -5624, -206, -1238, -482, -1052, -1617]
        # length_sorted_idx => [2 3 4 0 9 6 8 7 1 5]

    # sort dataframe
    df = df.iloc[length_sorted_idx]
    
    # calc max_len per batch
    sorted_input_length = df['input_lengths'].values
        # sorted_input_length => [6505 6223 5624 3001 1617 1238 1052  482  213  206]
    batch_max_length = np.zeros_like(sorted_input_length)
        # zeros_like; Return an array of zeros with the same shape and type as a given array.
        # batch_max_length => [0 0 0 0 0 0 0 0 0 0]
    total_iter = len(sorted_input_length) // batch_size if len(sorted_input_length) % batch_size == 0 else (len(sorted_input_length) // batch_size) + 1
        # total_iter => 5
    for i in range(total_iter):
        batch_max_length[i * batch_size:(i + 1) * batch_size] = np.max(sorted_input_length[i * batch_size:(i + 1) * batch_size])
    
    df['batch_max_length'] = batch_max_length
        # batch_max_length => [6505 6505 5624 5624 1617 1617 1052 1052  213  213]  
        # 6505; maximum number of words in corresponding row id. 
    return df

In [15]:
def get_score(df, masks, rank_pred, code_df_valid):
    #     df['cell_id2'] = [[y[i] for i in range(len(x)) if x[i] == 1] for x, y in
    #                           zip(df['cell_type'].values, df['cell_id'].values)]
    df['cell_id2'] = df['cell_id']
    df = df[['id', 'cell_id2']].explode('cell_id2')
        # df.head(2) =>
        #                    id  cell_id2
        # 0      0001bdd4021779  3fdc37be
        # 0      0001bdd4021779  073782ca

        # pd.isnull(df['cell_id2']) =>
        # 0       False
        # 0       False
        #         ...
        # Name: cell_id2, Length: 52282, dtype: bool
    df = df[~pd.isnull(df['cell_id2'])]
        # rank_pred.flatten()[0:10]
        # [-0.01354445  0.06333275  0.09860276  0.12805334  0.04778271  0.03880716 ...
        
        # masks.flatten()[0:10] =>
        # [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
    preds = rank_pred.flatten()[np.where(masks.flatten() == 1)]        
        # preds.flatten()[0:10]
        # [-0.01354445 -0.01515285 -0.01671559  0.00495259 -0.02522139 -0.02115827
    df['rank2'] = preds
        # df.groupby(by=['id', 'cell_id2'], as_index=False)['rank2'].head(2) => 
        # 0      -0.013544
        # 0      -0.015153
        #         ...
        # Name: rank2, Length: 47718, dtype: float64

        # ( showing all rows of id 0001bdd4021779 ), df.head(10) => 
        #                id  cell_id2     rank2
        # 0  0001bdd4021779  3fdc37be -0.013544
        # 0  0001bdd4021779  073782ca -0.015153
        # 0  0001bdd4021779  8ea7263c -0.016716
        # 0  0001bdd4021779  80543cd8  0.004953
        # 0  0001bdd4021779  38310c80 -0.025221
        # 0  0001bdd4021779  073e27e5 -0.021158
        # 0  0001bdd4021779  015d52a4  0.042571
        # 0  0001bdd4021779  ad7679ef  0.043002
        # 0  0001bdd4021779  07c52510 -0.010163
        # 0  0001bdd4021779  0a1a7a39  0.006047
        # 0  0001bdd4021779  0bcd3fef -0.015030
        # 0  0001bdd4021779  7fde4f04  0.083458
        # 0  0001bdd4021779  58bf360b  0.093474
    df = df.groupby(by=['id', 'cell_id2'], as_index=False)['rank2'].agg('mean')
        # df.groupby(by=['id', 'cell_id2'], as_index=True)['rank2'].agg('mean').head(10) =>
        # id              cell_id2
        # 0001bdd4021779  015d52a4    0.042571
        #                 073782ca   -0.015153
        #                 073e27e5   -0.021158
        #                 07c52510   -0.010163
        #                 0a1a7a39    0.006047
        #                 0bcd3fef   -0.015030
        #                 38310c80   -0.025221
        #                 3fdc37be   -0.013544
        #                 58bf360b    0.093474
        #                 7fde4f04    0.083458
        #                 80543cd8    0.004953
        #                 8ea7263c   -0.016716
        #                 ad7679ef    0.043002
        # Name: rank2, dtype: float64    
        
        # ( showing all rows of id 0001bdd4021779 ), df.head(10) =>
        #                 id  cell_id2     rank2
        # 0   0001bdd4021779  015d52a4  0.042571
        # 1   0001bdd4021779  073782ca -0.015153
        # 2   0001bdd4021779  073e27e5 -0.021158
        # 3   0001bdd4021779  07c52510 -0.010163
        # 4   0001bdd4021779  0a1a7a39  0.006047
        # 5   0001bdd4021779  0bcd3fef -0.015030
        # 6   0001bdd4021779  38310c80 -0.025221
        # 7   0001bdd4021779  3fdc37be -0.013544
        # 8   0001bdd4021779  58bf360b  0.093474
        # 9   0001bdd4021779  7fde4f04  0.083458
        
    df.rename(columns={'cell_id2': 'cell_id'}, inplace=True)
        # code_df_valid.head(2) =>
        #                id   cell_id     rank2
        # 0  00001756c60be8  1862f0a6  0.033333
        # 1  00001756c60be8  2a9e43d6  0.066667
    code_df_valid_tmp = code_df_valid[code_df_valid['id'].isin(df['id'])]
        # code_df_valid_tmp.head(2) =>    
        #                  id   cell_id     rank2
        # 102  0001bdd4021779  3fdc37be  0.090909
        # 103  0001bdd4021779  073782ca  0.181818    
    code_df_valid_tmp['rank3'] = code_df_valid_tmp.groupby(by=['id'])['rank2'].rank(ascending=True, method='first')
        # code_df_valid_tmp.head(3) =>
        #                  id   cell_id     rank2  rank3
        # 102  0001bdd4021779  3fdc37be  0.090909    1.0
        # 103  0001bdd4021779  073782ca  0.181818    2.0
        # 104  0001bdd4021779  8ea7263c  0.272727    3.0
        
    # df is much larger dataframe than code_df_valid_tmp, thus we find many empty values in ['rank3'] -
    # - column of "code_df_valid_tmp".
    tmp = code_df_valid_tmp[['id', 'cell_id', 'rank3']].merge(df, how='inner', on=['id', 'cell_id'])
        # tmp.head(3) =>    
        #                id   cell_id  rank3     rank2
        # 0  0001bdd4021779  3fdc37be    1.0 -0.013544
        # 1  0001bdd4021779  073782ca    2.0 -0.015153
        # 2  0001bdd4021779  8ea7263c    3.0 -0.016716    
        
    tmp['rank4'] = tmp.groupby(by=['id'])['rank2'].rank(ascending=True, method='first')
        # tmp.head(15) =>    
        #                 id   cell_id  rank3     rank2  rank4
        # 0   0001bdd4021779  3fdc37be    1.0 -0.013544    6.0
        # 1   0001bdd4021779  073782ca    2.0 -0.015153    4.0
        # 2   0001bdd4021779  8ea7263c    3.0 -0.016716    3.0
        # 3   0001bdd4021779  80543cd8    4.0  0.004953    8.0
        # 4   0001bdd4021779  38310c80    5.0 -0.025221    1.0
        # 5   0001bdd4021779  073e27e5    6.0 -0.021158    2.0
        # 6   0001bdd4021779  015d52a4    7.0  0.042571   10.0
        # 7   0001bdd4021779  ad7679ef    8.0  0.043002   11.0
        # 8   0001bdd4021779  07c52510    9.0 -0.010163    7.0
        # 9   0001bdd4021779  0a1a7a39   10.0  0.006047    9.0
        # 10  0001bdd4021779  0bcd3fef   11.0 -0.015030    5.0
        # 11  0002115f48f982  18281c6c    1.0 -0.024155    3.0
        # 12  0002115f48f982  e3b6b115    2.0 -0.072884    2.0

    tmp = tmp[['id', 'cell_id', 'rank3']].merge(tmp[['id', 'rank4', 'rank2']].rename(columns={'rank4': 'rank3'}),
                                                how='inner', on=['id', 'rank3'])
        # tmp.head(15) =>
        #                 id   cell_id  rank3     rank2
        # 0   0001bdd4021779  3fdc37be    1.0 -0.025221
        # 1   0001bdd4021779  073782ca    2.0 -0.021158
        # 2   0001bdd4021779  8ea7263c    3.0 -0.016716
        # 3   0001bdd4021779  80543cd8    4.0 -0.015153
        # 4   0001bdd4021779  38310c80    5.0 -0.015030
        # 5   0001bdd4021779  073e27e5    6.0 -0.013544
        # 6   0001bdd4021779  015d52a4    7.0 -0.010163
        # 7   0001bdd4021779  ad7679ef    8.0  0.004953
        # 8   0001bdd4021779  07c52510    9.0  0.006047
        # 9   0001bdd4021779  0a1a7a39   10.0  0.042571
        # 10  0001bdd4021779  0bcd3fef   11.0  0.043002
        # 11  0002115f48f982  18281c6c    1.0 -0.125229
        # 12  0002115f48f982  e3b6b115    2.0 -0.072884
        # 13  0002115f48f982  4a044c54    3.0 -0.024155
        # 14  0002115f48f982  365fe576    4.0 -0.014178
        
    tmp = tmp[['id', 'cell_id', 'rank2']]
    df = df.merge(tmp[['id', 'cell_id', 'rank2']].rename(columns={'rank2': 'rank3'}), how='left', on=['id', 'cell_id'])
    # now, ['rank3'] contains predictions for only code cells.
    # ['rank2'] contains predictions for both cell types.
        # df.head(4) =>
        #                id   cell_id     rank2     rank3
        # 0  0001bdd4021779  015d52a4  0.042571 -0.010163
        # 1  0001bdd4021779  073782ca -0.015153 -0.021158
        # 2  0001bdd4021779  073e27e5 -0.021158 -0.013544
        # 3  0001bdd4021779  07c52510 -0.010163  0.006047        
        
    
    df['rank2'] = np.where(pd.isnull(df['rank3']), df['rank2'], df['rank3']) 

    # df = pd.concat([df[['id', 'cell_id', 'rank2']], code_df_valid_tmp]).reset_index(drop=True)
    
    # sort ['id'] and then within each id sort ['rank2'].
    df = df.sort_values(by=['id', 'rank2'], ascending=True)
        # df.head(4) =>
        #                 id   cell_id     rank2     rank3
        # 7   0001bdd4021779  3fdc37be -0.025221 -0.025221
        # 1   0001bdd4021779  073782ca -0.021158 -0.021158
        # 11  0001bdd4021779  8ea7263c -0.016716 -0.016716
        # 10  0001bdd4021779  80543cd8 -0.015153 -0.015153
        
    return df

In [16]:
log_out_dir = os.path.join(parameter.result_dir, 'logs')
os.makedirs(log_out_dir, exist_ok=True)
log_dir = os.path.join(log_out_dir, '{}.txt'.format(args.model_name))
if os.path.exists(log_dir):
    os.remove(log_dir)
log = utils.Logger()
log.open(log_dir, mode='a')

model_dir = os.path.join(parameter.result_dir, '')
model_dir_1 = os.path.join(parameter.result_dir_1, '')
# os.makedirs(model_dir, exist_ok=True)

In [23]:
# set random seeds
seed_everything(parameter.random_seed)
os.environ["TOKENIZERS_PARALLELISM"] = "false"
log.write('>> predicting...\n')
start = time.time()

  
test_df = preprocess.read_json_data(mode='test')
test_df['rank'], test_df['fold_flag'] = 1,-1
test_df = preprocess.preprocess_df(test_df)
test_df = pd.concat(
        [test_df[test_df['cell_type'] == 0], test_df[test_df['cell_type'] == 1].sample(frac=1.0)]).reset_index(
        drop=True)
test_df['rank2'] = (test_df.groupby(by=['id', 'cell_type']).cumcount() + 1) / \
                    test_df.groupby(by=['id', 'cell_type'])['cell_id'].transform('count')
test_df.loc[test_df['cell_type'] == 1, 'rank2'] = -1
code_df_sub = test_df[test_df['cell_type'] == 0][['id', 'cell_id', 'rank2']].copy()
test_df = preprocess.get_truncated_df(test_df, cell_count=parameter.cell_count) 

# test_df = pd.read_parquet("../input/google-ai/train_df")[:1000]
# code_df_sub = pd.read_parquet("../input/google-ai/code_df_valid")

tokenizer = AutoTokenizer.from_pretrained(utils.get_model_path(args.model_name, res = parameter.model_dir))
test_df = get_sorted_test_df(test_df, 'source', tokenizer, batch_size=parameter.batch_size, cell_max_length=parameter.cell_max_length)

test_pre = test_df[test_df['batch_max_length'] <= 4096]
test_post = test_df[test_df['batch_max_length'] > 4096]

model = MarkdownModel(utils.get_model_path(args.model_name, res = parameter.model_dir), pretrained=True)
filename = os.path.join(model_dir_1, args.model_name + '_fold0.pth.tar')

# -------------------- part1
test_dataset = MarkdownDataset(test_pre, tokenizer, mode="test", parameter=parameter)
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=2,
                         num_workers=args.workers, drop_last=False, pin_memory=True)
y_preds, masks = get_preds(test_pre, test_loader, model, filename, max_length=4096+1024)


# -------------------- part2
if len(test_post) > 0:    
    parameter.set(**{'seq_max_length': 4096+1024}) 
    test_dataset = MarkdownDataset(test_post, tokenizer, mode="test", parameter=parameter)
    test_loader = DataLoader(test_dataset, shuffle=False, batch_size=1,
                             num_workers=args.workers, drop_last=False, pin_memory=True)
    y_preds2, masks2 = get_preds(test_post, test_loader, model, filename, max_length=4096+1024)
    y_preds = np.concatenate([y_preds2, y_preds])
    masks = np.concatenate([masks2, masks])

torch.cuda.empty_cache()
res = get_score(test_df, masks, y_preds, code_df_sub)
# res = pd.concat([res1, res2]).reset_index(drop=True)
# sub_df = res1
# sub_df = res1.sort_values(by=['id', 'rank2'], ascending=True)
sub_df = res.groupby(by=['id'], sort=False)['cell_id'].apply(lambda x: ' '.join(x)).reset_index()
sub_df.rename(columns={'cell_id': 'cell_order'}, inplace=True)
# sub_df['cell_order'] = sub_df['cell_order'].apply(lambda x: ' '.join(x.split()[::-1]))
# print(test_df.shape)
sub_df[['id', 'cell_order']].to_csv('submission.csv', index=False)

>> predicting...


100%|██████████| 4/4 [00:00<00:00, 162.63it/s]
../input/nar-ai4code-utils/preprocess.py:204: FutureWarning: The default value of regex will change from True to False in a future version.
  df['source'] = df['source'].str.replace("[SEP]", "")
../input/nar-ai4code-utils/preprocess.py:205: FutureWarning: The default value of regex will change from True to False in a future version.
  df['source'] = df['source'].str.replace("[CLS]", "")
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/opt/conda/lib/python3.7/site-packages/transformers/convert_slow_tokenizer.py:447: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens ma